In [1]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
# from sklearn.decomposition import PCA

import torch
from torchvision import transforms
from sklearn.utils import shuffle
from skimage.metrics import structural_similarity as ssim
import pennylane as qml
# from livelossplot import PlotLosses
import tensorflow as tf
from torch.utils.data import DataLoader, Dataset
from datetime import datetime
from sklearn.datasets import load_digits

%matplotlib inline

In [17]:
data = load_digits().data
classes = load_digits().target
filter_0 = np.where((classes == 0 ))
filter_1 = np.where((classes == 1 ))
# data, classes = data[filter], classes[filter]
# data, classes = shuffle(data, classes)
# data, classes = data[:45], classes[:45]
# print(data.shape, classes.shape)
num_images = 40
x_aux_0, y_0 = data[filter_0], classes[filter_0]
x_aux_0, y_0 = x_aux_0[:num_images], y_0[:num_images]
x_aux_1, y_1 = data[filter_1], classes[filter_1]
x_aux_1, y_1 = x_aux_1[:num_images], y_1[:num_images]
x_main = list(range(len(x_aux_0)+len(x_aux_1)))
for i in range(len(x_main)):
    x_main[i] = np.binary_repr(x_main[i], width=9)
    x_main[i] = [int(char) for char in x_main[i]]

x_main_0, x_main_1 = x_main[:num_images], x_main[num_images:]

x_aux_0_train, y_0_train, x_main_0_train = x_aux_0[:int(0.8*num_images)], y_0[:int(0.8*num_images)], x_main_0[:int(0.8*num_images)]
x_aux_1_train, y_1_train, x_main_1_train = x_aux_1[:int(0.8*num_images)], y_1[:int(0.8*num_images)], x_main_1[:int(0.8*num_images)]
x_aux_0_test, y_0_test, x_main_0_test = x_aux_0[int(0.8*num_images):], y_0[int(0.8*num_images):], x_main_0[int(0.8*num_images):]
x_aux_1_test, y_1_test, x_main_1_test = x_aux_1[int(0.8*num_images):], y_1[int(0.8*num_images):], x_main_1[int(0.8*num_images):]


# x_main = (x_main-np.mean(x_main))/np.std(x_main)
# x_aux = (x_aux -np.mean(x_aux))/np.std(x_aux)

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
BATCH_SIZE = 20

# DEFINE AUX PQC STRUCTURE AND DECLARE TORCHLAYER FOR IT
wires_aux_pqc = 6
layers_aux = 3
dev = qml.device("default.qubit", wires=9)

In [18]:
@qml.qnode(dev, interface='torch')
def aux_pqc(inputs, params):
    qml.AmplitudeEmbedding(features=inputs,normalize=True,wires=range(wires_aux_pqc))
    seed = 42
    qml.RandomLayers(weights=params, wires=range(wires_aux_pqc),seed=seed)
    return qml.probs(wires=range(wires_aux_pqc))

weights_aux_0 = {'params':(layers_aux, 20)}
weights_aux_1 = {'params':(layers_aux, 20)}

qcnn_aux_0 = qml.qnn.TorchLayer(aux_pqc, weights_aux_0, init_method=torch.nn.init.normal_)
qcnn_aux_1 = qml.qnn.TorchLayer(aux_pqc, weights_aux_1, init_method=torch.nn.init.normal_)

# DEFINE MAIN QRAM PQC STRUCTURE AND DECLARE TORCHLAYER FOR IT
wires_qcnn = 9
layers_qcnn = 3
@qml.qnode(dev, interface='torch')
def qcnn(inputs, params, weights):
    qml.AngleEmbedding(features=inputs, wires=range(wires_qcnn), rotation='Y')
    # qml.BasisEmbedding(features=inputs, wires=range(wires_qcnn))
    for l in range(layers_qcnn):
        cnt = 0
        #shape = qml.StronglyEntanglingLayers.shape(n_layers=layers_qcnn, n_wires=wires_qcnn)
        #weights = np.random.random(size=shape)
        for i in range(wires_qcnn - 1):
            qml.RY(params[l*36+cnt],wires=i)
            qml.RY(params[l*36+cnt+1],wires=i+1)
            qml.CNOT(wires=[i,i+1])
            qml.CRZ(params[l*36+cnt+2], wires=[i,i+1])
            qml.PauliX(wires=i+1)
            qml.CRX(params[l*36+cnt+3],wires=[i,i+1])
            cnt = cnt + 4
        for i in [wires_qcnn - 1]:
            qml.RY(params[l*36+cnt],wires=i)
            qml.RY(params[l*36+cnt+1],wires=i-(wires_qcnn - 1))
            qml.CNOT(wires=[i,i-(wires_qcnn - 1)])
            qml.CRZ(params[l*36+cnt+2], wires=[i,i-(wires_qcnn - 1)])
            qml.PauliX(wires=i-(wires_qcnn - 1))
            qml.CRX(params[l*36+cnt+3],wires=[i,i-(wires_qcnn - 1)])
            cnt = cnt + 4
    qml.StronglyEntanglingLayers(weights=weights, wires=range(wires_qcnn))
    return qml.probs(wires=[0,1,2,6,7,8])

weights_qcnn_0 = {"params":layers_qcnn*36, "weights":(layers_qcnn,wires_qcnn,layers_qcnn)}
weights_qcnn_1 = {"params":layers_qcnn*36, "weights":(layers_qcnn,wires_qcnn,layers_qcnn)}

qcnn_main_0 = qml.qnn.TorchLayer(qcnn, weights_qcnn_0, init_method=torch.nn.init.normal_)
qcnn_main_1 = qml.qnn.TorchLayer(qcnn, weights_qcnn_1, init_method=torch.nn.init.normal_)

# DEFINE OVERALL QRAM CLASS CONSISTING OF BOTH AUX PQC AND MAIN PQC
class QRAM(torch.nn.Module):
    def __init__(self, cls):
        super().__init__()
        self.cls = cls

        if self.cls == 0:
            self.qcnn_aux = torch.nn.Sequential(qcnn_aux_0)
            self.qcnn_main = torch.nn.Sequential(qcnn_main_0)
        elif self.cls == 1:
            self.qcnn_aux = torch.nn.Sequential(qcnn_aux_1)
            self.qcnn_main = torch.nn.Sequential(qcnn_main_1)
    
    def forward(self,x_aux,x_main):
        ground_truth = self.qcnn_aux(x_aux)
        pred = self.qcnn_main(x_main)
        return ground_truth, pred

qram_0 = QRAM(0).to(DEVICE).float()
qram_1 = QRAM(1).to(DEVICE).float()

loss_function = torch.nn.MSELoss()
# optimizer = torch.optim.SGD(qcnn_main.parameters(),lr=0.001,momentum=0.9,nesterov=True)
opt_0 = torch.optim.Adam(qram_0.parameters(),lr=0.01)
opt_1 = torch.optim.Adam(qram_1.parameters(),lr=0.01)

x_aux_0, x_aux_1 = torch.tensor(x_aux_0).to(DEVICE), torch.tensor(x_aux_1).to(DEVICE)
y_0, y_1 = torch.tensor(y_0,dtype=torch.double).to(DEVICE), torch.tensor(y_1,dtype=torch.double).to(DEVICE)
x_main_0, x_main_1 = torch.tensor(x_main_0, dtype=torch.float64).to(DEVICE), torch.tensor(x_main_1, dtype=torch.float64).to(DEVICE)

# TRAIN AUX PQC AND MAIN PQC TOGETHER
epochs = 20
for epoch in range(epochs):
    print("******EPOCH #" + str(epoch) + " START ******")
    epoch_loss_0 = torch.tensor([0], dtype=torch.float64).to(DEVICE)
    epoch_loss_1 = torch.tensor([0], dtype=torch.float64).to(DEVICE)
    for i in range(x_aux_0.shape[0]//BATCH_SIZE):
        bs_counter = 0
        opt_0.zero_grad()
        opt_1.zero_grad()
        loss_0 = torch.tensor([0], dtype=torch.float64, requires_grad=True).to(DEVICE)
        loss_1 = torch.tensor([0], dtype=torch.float64, requires_grad=True).to(DEVICE)
        while bs_counter < BATCH_SIZE: 
            # print(x_main[i*BATCH_SIZE+bs_counter])
            # ground_truth = aux_pqc(x_aux[i*BATCH_SIZE+bs_counter])
            # pred = qcnn_main(x_main[i*BATCH_SIZE+bs_counter])
            ground_truth_0, pred_0  = qram_0(x_aux_0[i*BATCH_SIZE+bs_counter],x_main_0[i*BATCH_SIZE+bs_counter])
            ground_truth_1, pred_1  = qram_1(x_aux_1[i*BATCH_SIZE+bs_counter],x_main_1[i*BATCH_SIZE+bs_counter])
            # print(pred.tolist())
            temp_loss_0 = torch.tensor([0], dtype=torch.float64, requires_grad=True).to(DEVICE)
            temp_loss_1 = torch.tensor([0], dtype=torch.float64, requires_grad=True).to(DEVICE)
            a = []
            for j in range(len(pred_0)):
                temp_loss_0 = temp_loss_0 + loss_function(pred_0[j], ground_truth_0[j])
                temp_loss_1 = temp_loss_1 + loss_function(pred_1[j], ground_truth_1[j])
                # a.append(torch.abs(pred[j]-ground_truth[j]).item())
            loss_0 = loss_0 + temp_loss_0
            loss_1 = loss_1 + temp_loss_1

            epoch_loss_0 = epoch_loss_0 + loss_0
            epoch_loss_1 = epoch_loss_1 + loss_1
            # loss = loss + loss_function(pred, y_train[i*BATCH_SIZE+bs_counter])
            # print(temp_loss.item())
            bs_counter = bs_counter + 1
        loss_0 = loss_0/BATCH_SIZE
        loss_1 = loss_1/BATCH_SIZE
        # loss.requires_grad = True
        loss_0.backward()
        loss_1.backward()
        # for param in qcnn_main.parameters():
        #     print(param.grad)
        opt_0.step()
        opt_1.step()
    print("EPOCH LOSS for 0:" + str(epoch_loss_0.item()))
    print("EPOCH LOSS for 1:" + str(epoch_loss_1.item()))
    print("******EPOCH #" + str(epoch) + " END ******")

print("****")
print()
print()
print()
print("****")

with torch.no_grad():
    for param in qram_0.parameters():
        param.requires_grad = False
        break
    for param in qram_1.parameters():
        param.requires_grad = False
        break

# TRAIN ONLY MAIN PQC AND KEEP TRAINED AUX PQC PARAMETERS FIXED
epochs = 20
for epoch in range(epochs):
    print("******EPOCH #" + str(epoch) + " START ******")
    epoch_loss_0 = torch.tensor([0], dtype=torch.float64).to(DEVICE)
    epoch_loss_1 = torch.tensor([0], dtype=torch.float64).to(DEVICE)
    for i in range(x_aux_0.shape[0]//BATCH_SIZE):
        bs_counter = 0
        opt_0.zero_grad()
        opt_1.zero_grad()
        loss_0 = torch.tensor([0], dtype=torch.float64, requires_grad=True).to(DEVICE)
        loss_1 = torch.tensor([0], dtype=torch.float64, requires_grad=True).to(DEVICE)
        while bs_counter < BATCH_SIZE: 
            # print(x_main[i*BATCH_SIZE+bs_counter])
            # ground_truth = aux_pqc(x_aux[i*BATCH_SIZE+bs_counter])
            # pred = qcnn_main(x_main[i*BATCH_SIZE+bs_counter])
            ground_truth_0, pred_0  = qram_0(x_aux_0[i*BATCH_SIZE+bs_counter],x_main_0[i*BATCH_SIZE+bs_counter])
            ground_truth_1, pred_1  = qram_1(x_aux_1[i*BATCH_SIZE+bs_counter],x_main_1[i*BATCH_SIZE+bs_counter])
            # print(pred.tolist())
            temp_loss_0 = torch.tensor([0], dtype=torch.float64, requires_grad=True).to(DEVICE)
            temp_loss_1 = torch.tensor([0], dtype=torch.float64, requires_grad=True).to(DEVICE)
            a = []
            for j in range(len(pred_0)):
                temp_loss_0 = temp_loss_0 + loss_function(pred_0[j], ground_truth_0[j])
                temp_loss_1 = temp_loss_1 + loss_function(pred_1[j], ground_truth_1[j])
                # a.append(torch.abs(pred[j]-ground_truth[j]).item())
            loss_0 = loss_0 + temp_loss_0
            loss_1 = loss_1 + temp_loss_1
            epoch_loss_0 = epoch_loss_0 + loss_0
            epoch_loss_1 = epoch_loss_1 + loss_1
            # loss = loss + loss_function(pred, y_train[i*BATCH_SIZE+bs_counter])
            # print(temp_loss.item())
            bs_counter = bs_counter + 1
        loss_0 = loss_0/BATCH_SIZE
        loss_1 = loss_1/BATCH_SIZE
        # loss.requires_grad = True
        loss_0.backward(retain_graph=True)
        loss_1.backward(retain_graph=True)
        opt_0.step()
        opt_1.step()
    print("EPOCH LOSS for 0:" + str(epoch_loss_0.item()))
    print("EPOCH LOSS for 1:" + str(epoch_loss_1.item()))
    print("******EPOCH #" + str(epoch) + " END ******")




******EPOCH #0 START ******
EPOCH LOSS for 0:14.9084672985389
EPOCH LOSS for 1:7.196406291031297
******EPOCH #0 END ******
******EPOCH #1 START ******
EPOCH LOSS for 0:11.663903457758355
EPOCH LOSS for 1:5.825408537742775
******EPOCH #1 END ******
******EPOCH #2 START ******
EPOCH LOSS for 0:8.937092218294664
EPOCH LOSS for 1:4.810793264797238
******EPOCH #2 END ******
******EPOCH #3 START ******
EPOCH LOSS for 0:6.819324052646263
EPOCH LOSS for 1:4.13909076500314
******EPOCH #3 END ******
******EPOCH #4 START ******
EPOCH LOSS for 0:5.337946143600788
EPOCH LOSS for 1:3.715137814715355
******EPOCH #4 END ******
******EPOCH #5 START ******
EPOCH LOSS for 0:4.404783494341645
EPOCH LOSS for 1:3.4091788721521112
******EPOCH #5 END ******
******EPOCH #6 START ******
EPOCH LOSS for 0:3.8504514079387357
EPOCH LOSS for 1:3.15026422457641
******EPOCH #6 END ******
******EPOCH #7 START ******
EPOCH LOSS for 0:3.5063037729827244
EPOCH LOSS for 1:2.925196364674009
******EPOCH #7 END ******
******E

In [19]:
x_aux_0_train, y_0_train, x_main_0_train = x_aux_0[:int(0.8*num_images)], y_0[:int(0.8*num_images)], x_main_0[:int(0.8*num_images)]
x_aux_1_train, y_1_train, x_main_1_train = x_aux_1[:int(0.8*num_images)], y_1[:int(0.8*num_images)], x_main_1[:int(0.8*num_images)]
x_aux_0_test, y_0_test, x_main_0_test = x_aux_0[int(0.8*num_images):], y_0[int(0.8*num_images):], x_main_0[int(0.8*num_images):]
x_aux_1_test, y_1_test, x_main_1_test = x_aux_1[int(0.8*num_images):], y_1[int(0.8*num_images):], x_main_1[int(0.8*num_images):]

# AUGMENT QNN CLASSIFIER TO THE QRAM AND TRAIN THE QNN CLASSIFIER BY KEEPING TRAINED QRAM PARAMETERS FIXED
layers_classifier = 5
@qml.qnode(dev, interface='torch')
def quantum_ram_classifier(inputs, params, weights, params2):
    qml.AngleEmbedding(features=inputs, wires=range(wires_qcnn), rotation='Y')
    # QRAM STRUCTURE
    for l in range(layers_qcnn):
        cnt = 0
        for i in range(wires_qcnn - 1):
            qml.RY(params[l*36+cnt],wires=i)
            qml.RY(params[l*36+cnt+1],wires=i+1)
            qml.CNOT(wires=[i,i+1])
            qml.CRZ(params[l*36+cnt+2], wires=[i,i+1])
            qml.PauliX(wires=i+1)
            qml.CRX(params[l*36+cnt+3],wires=[i,i+1])
            cnt = cnt + 4
        for i in [wires_qcnn - 1]:
            qml.RY(params[l*36+cnt],wires=i)
            qml.RY(params[l*36+cnt+1],wires=i-(wires_qcnn - 1))
            qml.CNOT(wires=[i,i-(wires_qcnn - 1)])
            qml.CRZ(params[l*36+cnt+2], wires=[i,i-(wires_qcnn - 1)])
            qml.PauliX(wires=i-(wires_qcnn - 1))
            qml.CRX(params[l*36+cnt+3],wires=[i,i-(wires_qcnn - 1)])
            cnt = cnt + 4
    qml.StronglyEntanglingLayers(weights=weights, wires=range(wires_qcnn))
    # QNN CLASSIFIER STRUCTURE
    for l in range(layers_classifier):
        cnt = 0
        for i in range(wires_qcnn - 1):
            qml.RY(params2[l*36+cnt],wires=i)
            qml.RY(params2[l*36+cnt+1],wires=i+1)
            qml.CNOT(wires=[i,i+1])
            qml.CRZ(params2[l*36+cnt+2], wires=[i,i+1])
            qml.PauliX(wires=i+1)
            qml.CRX(params2[l*36+cnt+3],wires=[i,i+1])
            cnt = cnt + 4
        for i in [wires_qcnn - 1]:
            qml.RY(params2[l*36+cnt],wires=i)
            qml.RY(params2[l*36+cnt+1],wires=i-(wires_qcnn - 1))
            qml.CNOT(wires=[i,i-(wires_qcnn - 1)])
            qml.CRZ(params2[l*36+cnt+2], wires=[i,i-(wires_qcnn - 1)])
            qml.PauliX(wires=i-(wires_qcnn - 1))
            qml.CRX(params2[l*36+cnt+3],wires=[i,i-(wires_qcnn - 1)])
            cnt = cnt + 4
    return qml.expval(qml.PauliZ(0))


weights_qram_classifier = {"params":layers_qcnn*36, "weights":(layers_qcnn,wires_qcnn,layers_qcnn), "params2":layers_classifier*36}

qram_classifier = qml.qnn.TorchLayer(quantum_ram_classifier, weights_qram_classifier, init_method=torch.nn.init.normal_)

optimizer_classifier = torch.optim.Adam(qram_classifier.parameters(),lr=0.01)

with torch.no_grad():
    cnt = 0
    for param in qram_classifier.parameters():
        if cnt < 2:
            param.requires_grad = False
        cnt = cnt + 1

epochs = 20
for epoch in range(epochs):
    flag = 1
    print("******EPOCH #" + str(epoch) + " START ******")
    epoch_loss = torch.tensor([0], dtype=torch.float64).to(DEVICE)
    running_acc = 0
    testing_acc = 0
    cnt_0 = 0
    cnt_1 = 0
    for i in range((x_aux_0_train.shape[0]+x_aux_1_train.shape[0])//BATCH_SIZE):
        bs_counter = 0
        optimizer_classifier.zero_grad()
        loss = torch.tensor([0], dtype=torch.float64, requires_grad=True).to(DEVICE)
        while bs_counter < BATCH_SIZE: 
            temp_loss = torch.tensor(0, dtype=torch.float64, requires_grad=True).to(DEVICE)
            if flag == 0:
                with torch.no_grad():
                    cnt = 0
                    for param in qram_classifier.parameters():
                        if cnt >= 2:
                            break
                        param = list(qram_0.parameters())[cnt]  
                        param.requires_grad = False  
                        cnt = cnt + 1
                loss = loss + loss_function((qram_classifier(x_main_0_train[cnt_0])+1.0)/2, y_0_train[cnt_0])
                temp_loss = loss_function((qram_classifier(x_main_0_train[cnt_0])+1.0)/2, y_0_train[cnt_0])
                # print(qram_classifier(x_main_0_train[cnt_0]).item(), (qram_classifier(x_main_0_train[cnt_0]).item()+1.0)/2, y_0_train[cnt_0].item())
                cnt_0 = cnt_0 + 1
            else:
                with torch.no_grad():
                    cnt = 0
                    for param in qram_classifier.parameters():
                        if cnt >= 2:
                            break
                        param = list(qram_1.parameters())[cnt] 
                        param.requires_grad = False   
                        cnt = cnt + 1            
                loss = loss + loss_function((qram_classifier(x_main_1_train[cnt_1])+1.0)/2, y_1_train[cnt_1])
                temp_loss = loss_function((qram_classifier(x_main_1_train[cnt_1])+1.0)/2, y_1_train[cnt_1])
                # print(qram_classifier(x_main_1_train[cnt_1]).item(), (qram_classifier(x_main_1_train[cnt_1]).item()+1.0)/2, y_1_train[cnt_1].item())
                cnt_1 = cnt_1 + 1
            # print(out.item(), y[i*BATCH_SIZE+bs_counter].item())
            epoch_loss = epoch_loss + loss
            if temp_loss.item() < 0.25:
                running_acc = running_acc + 1
            bs_counter = bs_counter + 1
            flag = 1-flag
        loss = loss/BATCH_SIZE
        loss.backward()
        optimizer_classifier.step()
        if i % 1 == 0 and i != 0:
            print("Running Accuracy: " + str(running_acc/((i+1)*BATCH_SIZE)*100) + "%")
    cnt_0 = 0
    cnt_1 = 0
    for i in range(x_aux_0_test.shape[0]+x_aux_1_test.shape[0]):
        if i%2==0:
            print(loss_function((qram_classifier(x_main_0_test[cnt_0])+1.0)/2, y_0_test[cnt_0]).item())
            if loss_function((qram_classifier(x_main_0_test[cnt_0])+1.0)/2, y_0_test[cnt_0]).item() < 0.25:
                testing_acc = testing_acc + 1
            cnt_0 = cnt_0 + 1    
        else:
            print(loss_function((qram_classifier(x_main_1_test[cnt_1])+1.0)/2, y_1_test[cnt_1]).item())
            if loss_function((qram_classifier(x_main_1_test[cnt_1])+1.0)/2, y_1_test[cnt_1]).item() < 0.25: 
                testing_acc = testing_acc + 1
            cnt_1 = cnt_1 + 1
        
    print("Running Accuracy: " + str(running_acc), "Total:" + str(x_aux_0_train.shape[0]+x_aux_1_train.shape[0]))
    print("Testing Accuracy: " + str(testing_acc) + "/" + str(x_aux_0_test.shape[0]+x_aux_1_test.shape[0]))
    print("EPOCH LOSS:" + str(epoch_loss.item()))
    print("******EPOCH #" + str(epoch) + " END ******")


******EPOCH #0 START ******
Running Accuracy: 30.0%
Running Accuracy: 36.666666666666664%
0.23872204243072698
0.2796815236585071
0.24708049569360846
0.24668722543179983
0.22386773132751972
0.2833725563769177
0.2361179943352505
0.24751527491849742
0.2581197439890497
0.2548026345649734
0.2652288859997436
0.2347286341992053
0.2540138132017502
0.2617054302610723
0.26032937806120227
0.2375517657751416
Running Accuracy: 22 Total:64
Testing Accuracy: 8/36
EPOCH LOSS:162.07791598784243
******EPOCH #0 END ******
******EPOCH #1 START ******
Running Accuracy: 55.00000000000001%
Running Accuracy: 55.00000000000001%
0.24741322762795484
0.28024199675231964
0.2546319985371585
0.24969955628852394
0.23359199295762656
0.2841429337247328
0.2466186927174623
0.24948281898806843
0.2642272764288999
0.2604356449561073
0.2698811454096118
0.2439638224309039
0.26154884953248764
0.2652360280981747
0.26861565753960154
0.24458020103507716
Running Accuracy: 33 Total:64
Testing Accuracy: 7/36
EPOCH LOSS:156.991388713

In [ ]:
print(list(qram_0.parameters())[0])